### Задание 1
POS Tagger

In [1]:
import pyconll
import nltk
from nltk.corpus import brown
from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger
from nltk.tag import BigramTagger, TrigramTagger
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
from gensim.models import Word2Vec
import pymorphy2
import numpy as np
import tensorflow as tf
import tqdm
import re
import string

from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Activation, Dense, Dot, Embedding, Flatten, GlobalAveragePooling1D, Reshape
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.python.eager import context

tf.config.threading.set_inter_op_parallelism_threads(16)
_ = tf.Variable([1])

/home/postas/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
!mkdir datasets
!wget -O ./datasets/ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
!wget -O ./datasets/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

--2021-04-12 17:38:33--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81043533 (77M) [text/plain]
Saving to: ‘./datasets/ru_syntagrus-ud-train.conllu’

./datasets/ru_synta 100%[===================>]  77,29M  4,70MB/s    in 18s     

2021-04-12 17:38:54 (4,31 MB/s) - ‘./datasets/ru_syntagrus-ud-train.conllu’ saved [81043533/81043533]

--2021-04-12 17:38:54--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercon

In [2]:
full_train = pyconll.load_from_file('datasets/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('datasets/ru_syntagrus-ud-dev.conllu')

In [3]:
fdata_train = [[(token.form, token.upos) for token in sent] for sent in full_train]
fdata_test = [[(token.form, token.upos) for token in sent] for sent in full_test]
fdata_sent_test = [[token.form for token in sent] for sent in full_test]

In [4]:
# N-Gram тэггер
default_tagger = nltk.DefaultTagger('NOUN')
unigram_tagger = nltk.UnigramTagger(fdata_train, backoff=default_tagger)
bigram_tagger = nltk.BigramTagger(fdata_train, backoff=unigram_tagger)
ngram_tagger = nltk.TrigramTagger(fdata_train, backoff=bigram_tagger)

In [5]:
display(ngram_tagger.tag(fdata_sent_test[1]), ngram_tagger.evaluate(fdata_test))

[('В', 'ADP'),
 ('старой', 'ADJ'),
 ('трактовке', 'NOUN'),
 ('вместо', 'ADP'),
 ('слова', 'NOUN'),
 ('"', 'PUNCT'),
 ('порядок', 'NOUN'),
 ('"', 'PUNCT'),
 ('использовалось', 'VERB'),
 ('слово', 'NOUN'),
 ('"', 'PUNCT'),
 ('последовательность', 'NOUN'),
 ('"', 'PUNCT'),
 (',', 'PUNCT'),
 ('но', 'CCONJ'),
 ('по', 'ADP'),
 ('мере', 'NOUN'),
 ('развития', 'NOUN'),
 ('параллельности', 'NOUN'),
 ('в', 'ADP'),
 ('работе', 'NOUN'),
 ('компьютеров', 'NOUN'),
 ('слово', 'NOUN'),
 ('"', 'PUNCT'),
 ('последовательность', 'NOUN'),
 ('"', 'PUNCT'),
 ('стали', 'VERB'),
 ('заменять', 'VERB'),
 ('более', 'ADV'),
 ('общим', 'ADJ'),
 ('словом', 'NOUN'),
 ('"', 'PUNCT'),
 ('порядок', 'NOUN'),
 ('"', 'PUNCT'),
 ('.', 'PUNCT')]

0.9120159741178849

In [6]:
# Эмм... мы уверены, что хотим еще что-то с этим сделать? Улучшить бейзлайн, который и так выше 91%? Really?
# Ну окей, посмотрим хотя-бы какие методы сколько дали скора тут:
default_tagger.evaluate(fdata_test)

0.23568564014423887

In [7]:
unigram_tagger.evaluate(fdata_test)

0.9063037104438378

In [8]:
bigram_tagger.evaluate(fdata_test)

0.9118980217706333

In [9]:
ngram_tagger.evaluate(fdata_test)

0.9120159741178849

In [10]:
# Специально прочекал - я не проверяю скор на трейне... на трейне он естественно еще выше
ngram_tagger.evaluate(fdata_train)

0.9835025002122714

In [61]:
# Мне кажется тут можно спокойно завернуть это в прод и идти спать ))
# В жизни конечно могут встретиться другие тексты, с другими формами этих слов - и вот тут скор упадет.
# Но у нас то сейчас их нет (размеченных) - не на чем провериться ну и ДЗ есть ДЗ.. 
# Приступим

In [12]:
all_train_texts = [' '.join(token.form for token in sent) for sent in full_train]
all_test_texts = [' '.join(token.form for token in sent) for sent in full_test]

In [13]:
all_train_labels = [' '.join(token.upos if token.upos else 'UNK' for token in sent) for sent in full_train]
all_test_labels = [' '.join(token.upos if token.upos else 'UNK' for token in sent) for sent in full_test]

In [14]:
print('\n'.join(all_train_labels[:10]))

NOUN PUNCT
NOUN ADJ NOUN NOUN PROPN PROPN AUX NOUN ADJ PUNCT VERB ADP NOUN ADV ADV PUNCT VERB ADP NOUN ADP NOUN CCONJ ADV PART VERB ADP NOUN NOUN ADP NOUN PUNCT NOUN PUNCT PUNCT
ADP NOUN PRON ADP NOUN VERB NOUN PUNCT PUNCT PRON ADP ADJ NOUN PUNCT CCONJ PRON PART ADP DET PUNCT PRON ADV ADV AUX VERB ADP ADJ NOUN PUNCT PART VERB PROPN PROPN PUNCT
ADV NOUN NOUN PROPN PROPN VERB ADP PRON PUNCT SCONJ VERB DET VERB CCONJ ADV VERB ADP NOUN PUNCT
NOUN AUX VERB ADV PUNCT CCONJ ADV PUNCT
ADP NOUN VERB NOUN NOUN PUNCT ADP NOUN PUNCT VERB NOUN ADP ADJ NOUN PUNCT
ADP NOUN VERB NOUN CCONJ VERB NOUN ADP NOUN VERB CCONJ ADV ADP PRON PUNCT SCONJ VERB NOUN NOUN PUNCT VERB ADP NOUN PUNCT ADV SCONJ PUNCT ADV PUNCT ADP NOUN VERB VERB PART ADJ NOUN PUNCT
NOUN VERB NOUN PUNCT ADJ PROPN PROPN PUNCT
ADP NOUN VERB ADJ ADJ NOUN ADP ADJ NOUN CCONJ ADP PRON NUM ADJ NOUN PUNCT
ADV VERB NOUN ADP NOUN PUNCT ADJ PUNCT VERB ADJ NOUN CCONJ ADP NUM NOUN ADV VERB NOUN PUNCT


In [15]:
# Так, ну нам то нужны списки
train_tokens = [pair[0] for pair in [item for sublist in fdata_train for item in sublist]]
train_labels = [pair[1] if pair[1] else 'UNK' for pair in [item for sublist in fdata_train for item in sublist]]
test_tokens = [pair[0] for pair in [item for sublist in fdata_test for item in sublist]]
test_labels = [pair[1] if pair[1] else 'UNK' for pair in [item for sublist in fdata_test for item in sublist]]

In [16]:
# Ну, для начала бейзлайн в виде LogReg..
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_labels)
test_enc_labels = le.transform(test_labels)

In [17]:
hvectorizer = HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=10000)

In [18]:
X_train = hvectorizer.fit_transform(train_tokens)

In [19]:
X_test = hvectorizer.transform(test_tokens)

In [20]:
X_train.shape

(871526, 10000)

In [21]:
lr = LogisticRegression(random_state=0, max_iter=100)
lr.fit(X_train, train_enc_labels)

/home/postas/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

In [22]:
pred = lr.predict(X_test)

In [23]:
accuracy_score(test_enc_labels, pred)

0.9133218751053146

In [24]:
# Ну - хотя... беру свои слова назад. Если по-агрессивнее себя вести (hash словарь 10К+ и сотни итераций регрессии)
# Вполне можно побить результат nltk-шных n-gram-ных алгоритмов
# Но - будем пробовать умные методы - не все же брут-форсом решать...

In [25]:
tfvectorizer = TfidfVectorizer(ngram_range=(1, 4), analyzer='char')

In [26]:
X_train_tf = tfvectorizer.fit_transform(train_tokens)
X_test_tf = tfvectorizer.transform(test_tokens)
X_train_tf.shape

(871526, 60615)

In [54]:
%%time
lr = LogisticRegression(random_state=0, max_iter=100)
lr.fit(X_train_tf, train_enc_labels)

CPU times: user 4min 41s, sys: 1min 53s, total: 6min 34s
Wall time: 1min 26s


/home/postas/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

In [55]:
pred_tf = lr.predict(X_test_tf)

In [56]:
accuracy_score(test_enc_labels, pred_tf)

0.9371903750884643

In [57]:
# Налицо!

In [62]:
lgbm = LGBMClassifier(n_estimators=100, learning_rate=0.087, reg_lambda=0.15, 
                      device="gpu", gpu_use_dp=False, max_bin=50)

In [63]:
%%time
lgbm.fit(X_train_tf, train_enc_labels, eval_set=[(X_test_tf, test_enc_labels)])

[1]	valid_0's multi_logloss: 1.58974
[2]	valid_0's multi_logloss: 1.36151
[3]	valid_0's multi_logloss: 1.20941
[4]	valid_0's multi_logloss: 1.09007
[5]	valid_0's multi_logloss: 0.996052
[6]	valid_0's multi_logloss: 0.916222
[7]	valid_0's multi_logloss: 0.846777
[8]	valid_0's multi_logloss: 0.789336
[9]	valid_0's multi_logloss: 0.739032
[10]	valid_0's multi_logloss: 0.69438
[11]	valid_0's multi_logloss: 0.655772
[12]	valid_0's multi_logloss: 0.621252
[13]	valid_0's multi_logloss: 0.591626
[14]	valid_0's multi_logloss: 0.565632
[15]	valid_0's multi_logloss: 0.541409
[16]	valid_0's multi_logloss: 0.519939
[17]	valid_0's multi_logloss: 0.500147
[18]	valid_0's multi_logloss: 0.482608
[19]	valid_0's multi_logloss: 0.466642
[20]	valid_0's multi_logloss: 0.451675
[21]	valid_0's multi_logloss: 0.438065
[22]	valid_0's multi_logloss: 0.425893
[23]	valid_0's multi_logloss: 0.414507
[24]	valid_0's multi_logloss: 0.403963
[25]	valid_0's multi_logloss: 0.394127
[26]	valid_0's multi_logloss: 0.385128


LGBMClassifier(device='gpu', gpu_use_dp=False, learning_rate=0.087, max_bin=50,
               reg_lambda=0.15)

In [64]:
lgbm_pred = lgbm.predict(X_test_tf)

In [65]:
accuracy_score(test_enc_labels, lgbm_pred)

0.929119064469383

In [67]:
# Более тяжелая бустинговая модель даже хуже справляется с этой задачей... либо ее вдумчиво тюнить надо еще

In [68]:
# Попробуем эмбеддинги

In [76]:
lemmer = pymorphy2.MorphAnalyzer()

In [95]:
def token_prep(token):
    return lemmer.parse(token.lower())[0].normal_form

In [77]:
train_sentences = [[token_prep(token.form) for token in sent] for sent in full_train]

In [81]:
train_sentences[:2]

[['анкета', '.'],
 ['начальник',
  'областной',
  'управление',
  'связь',
  'семён',
  'еремеевич',
  'быть',
  'человек',
  'простой',
  ',',
  'приходить',
  'на',
  'работа',
  'всегда',
  'вовремя',
  ',',
  'здороваться',
  'с',
  'секретарша',
  'за',
  'рука',
  'и',
  'иногда',
  'даже',
  'писать',
  'в',
  'стенгазета',
  'заметка',
  'под',
  'псевдоним',
  '"',
  'муха',
  '"',
  '.']]

In [82]:
# Неясно что делать с пунктуацией. Поидее в процессе выучивания эмбеддингов она участвовать не должна
# Но если ее убрать - на нее не будет эмбеддингов и классификатор не будет знать что с этими знаками делать

In [83]:
wv = Word2Vec(sentences=train_sentences, vector_size=500, window=5, min_count=1, workers=16)

In [93]:
wv.wv['.']

array([ 0.21278732,  0.6149245 ,  0.19744252, -0.27500916,  0.03706319,
       -0.08265077, -0.22061466,  0.5312531 , -0.03850966, -0.12585573,
       -0.21457656,  0.17632453, -0.01801084, -0.5097521 , -0.33300465,
        0.04141767, -0.04545579, -0.15542306,  0.10994256, -0.24727948,
        0.00395595,  0.23952244,  0.37681627, -0.06451444, -0.18628356,
       -0.27167365, -0.28647184, -0.38015452, -0.1577973 ,  0.21566355,
        0.5575756 ,  0.04229384, -0.21567576, -0.20044881, -0.14605278,
        0.41565326,  0.1468013 , -0.31206942, -0.00506167, -0.27831793,
        0.307549  ,  0.1254016 ,  0.06283051, -0.3811765 , -0.20665228,
       -0.07548293,  0.274045  ,  0.04755214, -0.3398731 ,  0.2926576 ,
       -0.12400011, -0.1232708 , -0.20995376, -0.25031757,  0.22504409,
       -0.18754041,  0.18352395,  0.1415958 , -0.02140302,  0.01020835,
        0.06774039, -0.33364186,  0.09648377,  0.16614659,  0.18396562,
       -0.12619042,  0.07967508,  0.05287063, -0.16100004,  0.23

In [91]:
# Окей, эмбеддинги есть. Даже на пунктуацию - попробуем классифицировать

In [96]:
train_tokens_emb = [wv.wv[token_prep(pair[0])] for pair in [item for sublist in fdata_train for item in sublist]]

In [100]:
%%time
lr = LogisticRegression(random_state=0, max_iter=100)
lr.fit(train_tokens_emb, train_enc_labels)

CPU times: user 33min 13s, sys: 6min 17s, total: 39min 30s
Wall time: 3min 1s


/home/postas/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

In [101]:
test_tokens_emb = [wv.wv[token_prep(pair[0])] for pair in [item for sublist in fdata_test for item in sublist]]

KeyError: "Key 'хорезми' not present"

In [103]:
# Таак.. напоролись 
# Даже не знаю, как тут быть.. выкидывать слова из теста поидее нельзя - что, если у нас на инференсе попадется
# Слово для которого нет эмбеддингов - мы же не сможем его не проклассифицировать. По аналогии с default_tagging
# Надо бы его просто обозвать существительным наверное.. В этом случае его во входном потоке можно просто заменять
# На эмбеддинг какого нибудь гарантированного существительного. (надеюсь я не слишком бредово мыслю в столь поздний час)

In [104]:
test_tokens_emb = [wv.wv[token_prep(pair[0])] if token_prep(pair[0]) in wv.wv \
                   else wv.wv['анкета'] for pair in [item for sublist in fdata_test for item in sublist]]

In [106]:
pred_emb = lr.predict(test_tokens_emb)

In [107]:
accuracy_score(test_enc_labels, pred_emb)

0.7940383513631921

In [108]:
# Нууу... такое.. открытым остается вопрос насчет пунктуации. Попробуем ее убрать из расчета эмбеддингов, но 
# Классифаеру ее отдадим в виде какого-нибудь специально сконструированного эмбеддинга типа все -1 например..

In [110]:
train_sentences = [[token_prep(token.form) for token in sent if token.form.isalpha()] for sent in full_train]

In [111]:
train_sentences[:2]

[['анкета'],
 ['начальник',
  'областной',
  'управление',
  'связь',
  'семён',
  'еремеевич',
  'быть',
  'человек',
  'простой',
  'приходить',
  'на',
  'работа',
  'всегда',
  'вовремя',
  'здороваться',
  'с',
  'секретарша',
  'за',
  'рука',
  'и',
  'иногда',
  'даже',
  'писать',
  'в',
  'стенгазета',
  'заметка',
  'под',
  'псевдоним',
  'муха']]

In [112]:
wv = Word2Vec(sentences=train_sentences, vector_size=500, window=5, min_count=1, workers=16)

In [114]:
'.' in wv.wv

False

In [126]:
# Отлично. Теперь эмбеддинг для пунктуации
wv.wv['анкета'].shape

(500,)

In [122]:
punct = np.zeros(500)

In [124]:
punct.shape

(500,)

In [127]:
# Заменяем
train_tokens_emb = [wv.wv[token_prep(pair[0])] if pair[0].isalpha() else punct \
                    for pair in [item for sublist in fdata_train for item in sublist]]

In [128]:
%%time
lr = LogisticRegression(random_state=0, max_iter=100)
lr.fit(train_tokens_emb, train_enc_labels)

CPU times: user 32min 8s, sys: 6min 9s, total: 38min 17s
Wall time: 2min 56s


/home/postas/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

In [129]:
# Так... Теперь еще сложнее процедура формирования теста ))

In [131]:
test_tokens_emb = [wv.wv[token_prep(pair[0])] if token_prep(pair[0]) in wv.wv \
                   else wv.wv['анкета'] if pair[0].isalpha() else punct \
                   for pair in [item for sublist in fdata_test for item in sublist]]

In [132]:
pred_emb = lr.predict(test_tokens_emb)

In [133]:
accuracy_score(test_enc_labels, pred_emb)

0.7639942034846493

In [134]:
# Еще хуже..

In [135]:
# Ну - для очистки совести еще нейросетевые варианты..

### Сетевой классификатор

In [ ]:
# Генерировать эмбеддинги сетями мне не очень понятно как.. в тех примерах которые мы рассматривали на вебинарах
# эмбеддинги были как-бы "побочным" продуктом процесса классификации текстов.
# В этой задаче не понятно - на что мы можем классифицировать тексты - лейблов у нас нет для текстов
# Есть только для слов (наши POS теги)
# Была дурная мысль раздать текстам рандомные или одинаковые таргеты, но каково будет качество результата 
# Такой мусорной классификации - остается под вопросом. Думаю не гуд идея.

# Поэтому будем брать уже векторизованные слова (наши tfidf или hashing) и тренировать сеть на классификацию.

In [17]:
# Сколько у нас лейблов
len(list(set(train_enc_labels)))

18

In [18]:
list(set(test_enc_labels))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]

In [19]:
tfvectorizer = TfidfVectorizer(ngram_range=(1, 4), analyzer='char', max_features=1000)

In [20]:
X_train_tf = tfvectorizer.fit_transform(train_tokens)

In [21]:
X_test_tf = tfvectorizer.transform(test_tokens)

In [22]:
model = Sequential([
    Dense(1000, activation='relu'),
    Dense(128),
    Dense(18, activation='softmax')])

In [23]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [24]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [27]:
model.fit(
    x = pd.DataFrame.sparse.from_spmatrix(X_train_tf),
    y = train_enc_labels,
    batch_size=64,
    validation_data=(pd.DataFrame.sparse.from_spmatrix(X_test_tf), test_enc_labels),
    epochs=15,
    callbacks=[tensorboard_callback])

Epoch 1/15
13618/13618 [==============================] - 55s 4ms/step - loss: 0.2929 - accuracy: 0.9039 - val_loss: 0.1754 - val_accuracy: 0.9360
Epoch 2/15
13618/13618 [==============================] - 54s 4ms/step - loss: 0.1351 - accuracy: 0.9496 - val_loss: 0.1611 - val_accuracy: 0.9403
Epoch 3/15
13618/13618 [==============================] - 55s 4ms/step - loss: 0.1180 - accuracy: 0.9551 - val_loss: 0.1577 - val_accuracy: 0.9431
Epoch 4/15
13618/13618 [==============================] - 55s 4ms/step - loss: 0.1103 - accuracy: 0.9574 - val_loss: 0.1557 - val_accuracy: 0.9448
Epoch 5/15
13618/13618 [==============================] - 55s 4ms/step - loss: 0.1049 - accuracy: 0.9592 - val_loss: 0.1569 - val_accuracy: 0.9451
Epoch 6/15
13618/13618 [==============================] - 55s 4ms/step - loss: 0.1026 - accuracy: 0.9598 - val_loss: 0.1550 - val_accuracy: 0.9460
Epoch 7/15
13618/13618 [==============================] - 54s 4ms/step - loss: 0.1004 - accuracy: 0.9607 - val_loss: 0

In [28]:
%load_ext tensorboard
%tensorboard --logdir logs

In [30]:
# Вобщем - 0.947 - отыграли один процент у ЛогРегрессии )))

In [38]:
tfvectorizer = TfidfVectorizer(ngram_range=(1, 4), analyzer='char', max_features=2048)

In [39]:
X_train_tf = tfvectorizer.fit_transform(train_tokens)

In [40]:
X_test_tf = tfvectorizer.transform(test_tokens)

In [41]:
model = Sequential([
    Dense(2048, activation='relu'),
    Dense(512),
    Dense(18, activation='softmax')])

In [42]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [43]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [44]:
model.fit(
    x = pd.DataFrame.sparse.from_spmatrix(X_train_tf),
    y = train_enc_labels,
    batch_size=128,
    validation_data=(pd.DataFrame.sparse.from_spmatrix(X_test_tf), test_enc_labels),
    epochs=15,
    callbacks=[tensorboard_callback])

Epoch 1/15
6809/6809 [==============================] - 33s 5ms/step - loss: 0.2588 - accuracy: 0.9151 - val_loss: 0.1539 - val_accuracy: 0.9456
Epoch 2/15
6809/6809 [==============================] - 31s 5ms/step - loss: 0.1163 - accuracy: 0.9565 - val_loss: 0.1496 - val_accuracy: 0.9482
Epoch 3/15
6809/6809 [==============================] - 31s 5ms/step - loss: 0.1022 - accuracy: 0.9606 - val_loss: 0.1397 - val_accuracy: 0.9506
Epoch 4/15
6809/6809 [==============================] - 31s 5ms/step - loss: 0.0946 - accuracy: 0.9631 - val_loss: 0.1397 - val_accuracy: 0.9510
Epoch 5/15
6809/6809 [==============================] - 31s 5ms/step - loss: 0.0896 - accuracy: 0.9645 - val_loss: 0.1442 - val_accuracy: 0.9491
Epoch 6/15
6809/6809 [==============================] - 31s 5ms/step - loss: 0.0879 - accuracy: 0.9653 - val_loss: 0.1390 - val_accuracy: 0.9528
Epoch 7/15
6809/6809 [==============================] - 31s 5ms/step - loss: 0.0855 - accuracy: 0.9657 - val_loss: 0.1359 - val_ac

In [45]:
%tensorboard --logdir logs

In [46]:
# Еще процент.. 0.95

In [63]:
preds = model.predict(pd.DataFrame.sparse.from_spmatrix(X_test_tf[:30]))

In [64]:
[(token, le.classes_[np.argmax(probs)]) for token, probs in zip(test_tokens[:30], preds)]

[('Алгоритм', 'NOUN'),
 (',', 'PUNCT'),
 ('от', 'ADP'),
 ('имени', 'NOUN'),
 ('учёного', 'NOUN'),
 ('аль', 'PROPN'),
 ('-', 'PUNCT'),
 ('Хорезми', 'NOUN'),
 (',', 'PUNCT'),
 ('-', 'PUNCT'),
 ('точный', 'ADJ'),
 ('набор', 'NOUN'),
 ('инструкций', 'NOUN'),
 (',', 'PUNCT'),
 ('описывающих', 'VERB'),
 ('порядок', 'NOUN'),
 ('действий', 'NOUN'),
 ('исполнителя', 'NOUN'),
 ('для', 'ADP'),
 ('достижения', 'NOUN'),
 ('результата', 'NOUN'),
 ('решения', 'NOUN'),
 ('задачи', 'NOUN'),
 ('за', 'ADP'),
 ('конечное', 'ADJ'),
 ('время', 'NOUN'),
 ('.', 'PUNCT'),
 ('В', 'ADP'),
 ('старой', 'ADJ'),
 ('трактовке', 'NOUN')]

In [65]:
# Слегка ломает, что модель не понимает контекста... так "достижение" и "решение" здесь явно глаголы - 
# но скорее всего они существительное даже в разметке... так, что этот факап модели даже не предьявишь.

In [66]:
# В общем с этой задачей на данном этапе закончу

### Задание 2 NER

In [2]:
import corus

In [3]:
!wget http://ai-center.botik.ru/Airec/ai-resources/Persons-1000.zip

--2021-04-13 15:17:17--  http://ai-center.botik.ru/Airec/ai-resources/Persons-1000.zip
Resolving ai-center.botik.ru (ai-center.botik.ru)... failed: Temporary failure in name resolution.
wget: unable to resolve host address ‘ai-center.botik.ru’


In [ ]:
# Интересно... что еще за новости ))

In [3]:
!wget ai-center.botik.ru/Airec/ai-resources/Persons-1000.zip

--2021-04-13 16:45:11--  http://ai-center.botik.ru/Airec/ai-resources/Persons-1000.zip
Resolving ai-center.botik.ru (ai-center.botik.ru)... 95.129.138.2
Connecting to ai-center.botik.ru (ai-center.botik.ru)|95.129.138.2|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3363777 (3,2M) [application/zip]
Saving to: ‘Persons-1000.zip.1’

Persons-1000.zip.1  100%[===================>]   3,21M  2,28MB/s    in 1,4s    

2021-04-13 16:45:13 (2,28 MB/s) - ‘Persons-1000.zip.1’ saved [3363777/3363777]



In [4]:
path = 'Persons-1000.zip'
records = corus.persons.load_persons(path)
rec = next(records)

In [5]:
rec

PersonsMarkup(
    text='Россия рассчитывает на конструктивное воздействие США на Грузию\r\n\r\n04/08/2008 12:08\r\n\r\nМОСКВА, 4 авг - РИА Новости. Россия рассчитывает, что США воздействуют на Тбилиси в связи с обострением ситуации в зоне грузино-осетинского конфликта. Об этом статс-секретарь - заместитель министра иностранных дел России Григорий Карасин заявил в телефонном разговоре с заместителем госсекретаря США Дэниэлом Фридом.\r\n\r\n"С российской стороны выражена глубокая озабоченность в связи с новым витком напряженности вокруг Южной Осетии, противозаконными действиями грузинской стороны по наращиванию своих вооруженных сил в регионе, бесконтрольным строительством фортификационных сооружений", - говорится в сообщении.\r\n\r\n"Россия уже призвала Тбилиси к ответственной линии и рассчитывает также на конструктивное воздействие со стороны Вашингтона", - сообщил МИД России. ',
    spans=[PersonsSpan(
         id=1,
         start=308,
         stop=324,
         value='ГРИГОРИЙ КАР

In [6]:
# Ну вобщем так... нужно сделать нормальный датасет с нормальной разметкой собственно. Для классификатора

In [7]:
rec.spans[1]

PersonsSpan(
    id=2,
    start=387,
    stop=402,
    value='ДЭНИЭЛ ФРИД'
)

In [8]:
# Кстати вот интересно, почему для этого текста не помечены также "Россия", "Северная Осетия", "США", "Тбилиси" и тп

In [9]:
def get_spans(text, spans):
    result = []
    for span in spans:
        result.extend(text[span.start:span.stop].split())
    return result

In [10]:
lemmer = pymorphy2.MorphAnalyzer()

In [11]:
def token_prep(token):
    return lemmer.parse(token.lower())[0].normal_form

In [12]:
get_spans(rec.text, rec.spans)

['Григорий', 'Карасин', 'Дэниэлом', 'Фридом']

In [14]:
# Теперь лабелинг текста
labelled_text = [(token_prep(token), 1) if token in get_spans(rec.text, rec.spans) else (token_prep(token), 0) \
 for token in re.findall(r"\w+|[^\w\s]", rec.text, re.UNICODE) if token.isalpha()]

In [15]:
labelled_text

[('россия', 0),
 ('рассчитывать', 0),
 ('на', 0),
 ('конструктивный', 0),
 ('воздействие', 0),
 ('сша', 0),
 ('на', 0),
 ('грузия', 0),
 ('москва', 0),
 ('авг', 0),
 ('риа', 0),
 ('новость', 0),
 ('россия', 0),
 ('рассчитывать', 0),
 ('что', 0),
 ('сша', 0),
 ('воздействовать', 0),
 ('на', 0),
 ('тбилиси', 0),
 ('в', 0),
 ('связь', 0),
 ('с', 0),
 ('обострение', 0),
 ('ситуация', 0),
 ('в', 0),
 ('зона', 0),
 ('грузино', 0),
 ('осетинский', 0),
 ('конфликт', 0),
 ('о', 0),
 ('это', 0),
 ('статс', 0),
 ('секретарь', 0),
 ('заместитель', 0),
 ('министр', 0),
 ('иностранный', 0),
 ('дело', 0),
 ('россия', 0),
 ('григорий', 1),
 ('карасиный', 1),
 ('заявить', 0),
 ('в', 0),
 ('телефонный', 0),
 ('разговор', 0),
 ('с', 0),
 ('заместитель', 0),
 ('госсекретарь', 0),
 ('сша', 0),
 ('дэниэлом', 1),
 ('фрид', 1),
 ('с', 0),
 ('российский', 0),
 ('сторона', 0),
 ('выразить', 0),
 ('глубокий', 0),
 ('озабоченность', 0),
 ('в', 0),
 ('связь', 0),
 ('с', 0),
 ('новый', 0),
 ('виток', 0),
 ('напряжё

In [16]:
[item for item in labelled_text if item[1] == 1]

[('григорий', 1), ('карасиный', 1), ('дэниэлом', 1), ('фрид', 1)]

In [17]:
# Ммм... что то с лемматизацией фигня получается. Может просто стемминг на имена собственные?

In [26]:
stemmer = nltk.stem.porter.PorterStemmer()

In [27]:
stemmer.stem(word='карасин')

'карасин'

In [28]:
stemmer.stem(word='дэниэлом')

'дэниэлом'

In [29]:
# Тоже не работает как хочется

In [30]:
nltk_lemmatizer = nltk.WordNetLemmatizer()

In [32]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/postas/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [33]:
nltk_lemmatizer.lemmatize('карасин')

'карасин'

In [34]:
nltk_lemmatizer.lemmatize('дэниэлом')

'дэниэлом'

In [41]:
lemmer.parse('Дэниэлом')[0].normal_form

'дэниэлом'

In [42]:
# Ничего путного не получается... возможно будем использовать форму которая в таргете у нас дана
# И надеяться на буквенные ngram-ы при векторизации (может зря мы его лемматизируем тогда...)

In [52]:
def get_spans(text, spans):
    raw_list = []
    for span in spans:
        raw_list.extend([{real_form: ref_form} for real_form, ref_form in \
                         zip(text[span.start:span.stop].split(), span.value.split())])
    result = {}
    for item in raw_list:
        result.update(item)
    return result

In [53]:
get_spans(rec.text, rec.spans)

{'Григорий': 'ГРИГОРИЙ',
 'Карасин': 'КАРАСИН',
 'Дэниэлом': 'ДЭНИЭЛ',
 'Фридом': 'ФРИД'}

In [57]:
labelled_text = [(get_spans(rec.text, rec.spans).get(token).lower(), 1) \
                 if token in get_spans(rec.text, rec.spans) else (token_prep(token), 0) \
                 for token in re.findall(r"\w+|[^\w\s]", rec.text, re.UNICODE) if token.isalpha()]

In [58]:
labelled_text

[('россия', 0),
 ('рассчитывать', 0),
 ('на', 0),
 ('конструктивный', 0),
 ('воздействие', 0),
 ('сша', 0),
 ('на', 0),
 ('грузия', 0),
 ('москва', 0),
 ('авг', 0),
 ('риа', 0),
 ('новость', 0),
 ('россия', 0),
 ('рассчитывать', 0),
 ('что', 0),
 ('сша', 0),
 ('воздействовать', 0),
 ('на', 0),
 ('тбилиси', 0),
 ('в', 0),
 ('связь', 0),
 ('с', 0),
 ('обострение', 0),
 ('ситуация', 0),
 ('в', 0),
 ('зона', 0),
 ('грузино', 0),
 ('осетинский', 0),
 ('конфликт', 0),
 ('о', 0),
 ('это', 0),
 ('статс', 0),
 ('секретарь', 0),
 ('заместитель', 0),
 ('министр', 0),
 ('иностранный', 0),
 ('дело', 0),
 ('россия', 0),
 ('григорий', 1),
 ('карасин', 1),
 ('заявить', 0),
 ('в', 0),
 ('телефонный', 0),
 ('разговор', 0),
 ('с', 0),
 ('заместитель', 0),
 ('госсекретарь', 0),
 ('сша', 0),
 ('дэниэл', 1),
 ('фрид', 1),
 ('с', 0),
 ('российский', 0),
 ('сторона', 0),
 ('выразить', 0),
 ('глубокий', 0),
 ('озабоченность', 0),
 ('в', 0),
 ('связь', 0),
 ('с', 0),
 ('новый', 0),
 ('виток', 0),
 ('напряжённос

In [59]:
[item for item in labelled_text if item[1] == 1]

[('григорий', 1), ('карасин', 1), ('дэниэл', 1), ('фрид', 1)]

In [60]:
# Так будет норм (если на векторизации мы сможем удачно порезать исходный текст на нграмы)

In [63]:
# Группируем все слова корпуса с признаками
all_tokens_labelled = []
for rec in records:
    all_tokens_labelled.append([(get_spans(rec.text, rec.spans).get(token).lower(), 1) \
                 if token in get_spans(rec.text, rec.spans) else (token_prep(token), 0) \
                 for token in re.findall(r"\w+|[^\w\s]", rec.text, re.UNICODE) if token.isalpha()])

In [66]:
len(all_tokens_labelled)

998

In [71]:
all_tokens_labelled[1]

[('октябрь', 0),
 ('назначить', 0),
 ('очередной', 0),
 ('выборы', 0),
 ('верховный', 0),
 ('совет', 0),
 ('аджарский', 0),
 ('ар', 0),
 ('по', 0),
 ('распоряжение', 0),
 ('президент', 0),
 ('грузия', 0),
 ('михаил', 1),
 ('саакашвили', 1),
 ('октябрь', 0),
 ('год', 0),
 ('назначить', 0),
 ('очередной', 0),
 ('выборы', 0),
 ('верховный', 0),
 ('совет', 0),
 ('аджарский', 0),
 ('ар', 0),
 ('о', 0),
 ('это', 0),
 ('новость', 0),
 ('грузия', 0),
 ('сообщить', 0),
 ('в', 0),
 ('пресс', 0),
 ('служба', 0),
 ('администрация', 0),
 ('президент', 0),
 ('в', 0),
 ('вторник', 0),
 ('выборы', 0),
 ('верховный', 0),
 ('совет', 0),
 ('аджарский', 0),
 ('автономный', 0),
 ('республика', 0),
 ('назначить', 0),
 ('в', 0),
 ('соответствие', 0),
 ('с', 0),
 ('ой', 0),
 ('статья', 0),
 ('и', 0),
 ('м', 0),
 ('пункт', 0),
 ('й', 0),
 ('статья', 0),
 ('конституционный', 0),
 ('закон', 0),
 ('грузия', 0),
 ('о', 0),
 ('статус', 0),
 ('аджарский', 0),
 ('автономный', 0),
 ('республика', 0)]

In [70]:
[item for item in all_tokens_labelled[1] if item[1] == 1]

[('михаил', 1), ('саакашвили', 1)]

In [76]:
# Все это бы объединить в один большой список (сейчас он резделен на документы)
joint_tokens_labbeled = [item for sublist in all_tokens_labelled for item in sublist]

In [77]:
len(joint_tokens_labbeled)

218858

In [81]:
# Делим на трейн и тест
train_tokens_labbeled = joint_tokens_labbeled[:150000]
test_tokens_labbeled = joint_tokens_labbeled[150000:]

In [82]:
# разбираем  на токены и таргеты
train_X = [pair[0] for pair in train_tokens_labbeled]
test_X = [pair[0] for pair in test_tokens_labbeled]
train_y = [pair[1] for pair in train_tokens_labbeled]
test_y = [pair[1] for pair in test_tokens_labbeled]

In [86]:
# Пока без эмбеддингов ))
tfvectorizer = TfidfVectorizer(ngram_range=(2, 5), analyzer='char')

In [87]:
train_X_tf = tfvectorizer.fit_transform(train_X)
test_X_tf = tfvectorizer.transform(test_X)
train_X_tf.shape

(150000, 54102)

In [89]:
# Штош.. попробуем простую модель

In [100]:
%%time
lr = LogisticRegression(random_state=0, max_iter=1000)
lr.fit(train_X_tf, train_y)

CPU times: user 17.1 s, sys: 18.4 s, total: 35.5 s
Wall time: 2.46 s


LogisticRegression(max_iter=1000, random_state=0)

In [101]:
pred = lr.predict(test_X_tf)

In [102]:
accuracy_score(test_y, pred)

0.9743820616340876

In [103]:
# Не знаю... может я что-то совершенно не понимаю - почему такой скор опять?!

In [ ]:
# Real life test

In [107]:
records = corus.persons.load_persons(path)

In [109]:
rec = next(records)

In [116]:
rec = next(records)

In [117]:
rec.spans

[PersonsSpan(
     id=1,
     start=320,
     stop=334,
     value='ЕГОР БОРИСОВ'
 ), PersonsSpan(
     id=2,
     start=989,
     stop=1003,
     value='ЕГОР БОРИСОВ'
 ), PersonsSpan(
     id=3,
     start=1318,
     stop=1332,
     value='ЕГОР БОРИСОВ'
 ), PersonsSpan(
     id=4,
     start=1624,
     stop=1643,
     value='АНАТОЛИЙ ПОДЛАСЕНКО'
 )]

In [118]:
rec.text

'Следственное управление при прокуратуре требует наказать премьера Якутии\r\n\r\nСледственное управление Следственного комитета при прокуратуре Российской Федерации по Якутии обжаловало решение прокуратуры республики. Ранее прокуратура отказала в возбуждении уголовного дела в отношении председателя правительства республики Егора Борисова, подозреваемого в хищении 30 млн руб.\r\n\r\nКак сообщили в четверг корреспонденту Агентства национальных новостей в следственном управлении, еще 16 мая 2007 г. прокуратурой Якутии было возбуждено уголовное дело № 66144 по признакам преступления, предусмотренного ч. 4 ст. 159 УК РФ по факту причинения имущественного ущерба в размере 30 млн руб. государственному унитарному предприятию <Дирекция по строительству железной дороги <Беркакит-Томмот-Якутск>.\r\n\r\n<В ходе расследования данного уголовного дела собраны достаточные данные, свидетельствующие о причастности к причинению ущерба данному предприятию председателя правительства Республики Саха (Якутия

In [125]:
text_prepared = [token_prep(token) for token in rec.text.split()]
text_vectorized = tfvectorizer.transform(text_prepared)
preds = lr.predict(text_vectorized)
result = []
for word, label in zip(rec.text.split(), preds):
    result.append(word if label == 0 else f'\x1b[31m{word}\x1b[0m')
print(' '.join(result))

Следственное управление при прокуратуре требует наказать премьера Якутии Следственное управление Следственного комитета при прокуратуре Российской Федерации по Якутии обжаловало решение прокуратуры республики. Ранее прокуратура отказала в возбуждении уголовного дела в отношении председателя правительства республики Егора Борисова, подозреваемого в хищении 30 млн руб. Как сообщили в четверг корреспонденту Агентства национальных новостей в следственном управлении, еще 16 мая 2007 г. прокуратурой Якутии было возбуждено уголовное дело № 66144 по признакам преступления, предусмотренного ч. 4 ст. 159 УК РФ по факту причинения имущественного ущерба в размере 30 млн руб. государственному унитарному предприятию <Дирекция по строительству железной дороги <Беркакит-Томмот-Якутск>. <В ходе расследования данного уголовного дела собраны достаточные данные, свидетельствующие о причастности к причинению ущерба данному предприятию председателя правительства Республики Саха (Якутия) Егора Борисова>, - г

In [126]:
# Не знаю, отобразиться ли цветная печать там где этот ноутбук будут просматривать - у меня в локальном юпитере
# Все 4 NER-а нашлись корректно и подсвечены красным )))

In [127]:
# Попозже еще поэкспериментирую с deeppavlov и natasha - сейчас пока-что залью в таком виде